In [1]:
# Imports:
from pyspark.sql import SparkSession, Row, Column
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import re
from bs4 import BeautifulSoup
from itertools import zip_longest

In [2]:
# Setup:
spark = (
    SparkSession
    .builder
    .config("spark.app.name", "MYC_jupyter_notebook_app")
    .config("spark.master", "spark://127.0.1.1:7077")
    .config("spark.jars", "../local/jars/hadoop-aws-3.4.1.jar, ../local/jars/bundle-2.28.26.jar")
    .getOrCreate()
)
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/06 19:20:50 WARN Utils: Your hostname, pop-os, resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp0s20f3)
25/09/06 19:20:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/09/06 19:20:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Extract data by crawling on Shizune website i.e. https://shizune.co/investors/investors-india
url = 'https://shizune.co/investors/investors-india'
response = requests.get(url)
html = response.text

soup = BeautifulSoup(html, 'html.parser')

investor_name_spans = soup.find_all("span", class_="investor__name")
investor_names = [investor_name_span.text for investor_name_span in investor_name_spans]

investor_meta_spans = soup.find_all("span", class_="investor__meta")
investor_metas = [investor_meta_span.text for investor_meta_span in investor_meta_spans]

investor_links = []
links_divs = soup.find_all(class_="links")
for link_div in links_divs:
    individual_links = []
    links = link_div.find_all("a")
    for link in links:
        individual_links.append(link.get("href"))
    investor_links.append(individual_links)

description_divs = soup.find_all("div", class_="desc")
investor_descs = [description_div.text for description_div in description_divs]

investment_focus_list = []
portfolio_highlights_list_with_links = []
portfolio_feature_list_divs = soup.find_all("div", class_="portfolio-feature-list")
for portfolio_feature_list_div in portfolio_feature_list_divs:
    if portfolio_feature_list_div.span.text.strip() == "Investment focus":
        investment_focuses = [list_item.text for list_item in portfolio_feature_list_div.ul.find_all("li")]
        investment_focus_list.append(investment_focuses)
    if portfolio_feature_list_div.span.text.strip() == "Portfolio highlights":
        portfolio_highlights = [list_item.a.get("href") + " — " + list_item.text for list_item in portfolio_feature_list_div.ul.find_all("li")]
        portfolio_highlights_list_with_links.append(portfolio_highlights)

In [4]:
# Create Dataframe from the data extracted above
schema = StructType([
    StructField('name',
                StringType(), True),
    StructField('meta',
                StringType(), True),
    StructField('links',
                ArrayType(StringType()), True),
    StructField('desc',
                StringType(), True),
    StructField('focus',
               ArrayType(StringType()), True),
    StructField('portfolio_highlights',
                ArrayType(StringType()), True),
])

df = spark.createDataFrame(list(zip_longest(
    investor_names,
    investor_metas,
    investor_links,
    investor_descs,
    investment_focus_list,
    portfolio_highlights_list_with_links,
    fillvalue=None
)), 
schema=schema)

df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                name|                meta|               links|                desc|               focus|portfolio_highlights|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Inflection Point ...|\n              V...|[https://www.link...|\n            Inf...|[Artificial Intel...|[https://kisah.in...|
|        LVX Ventures|\n              V...|[http://www.linke...|\n            Con...|[E-Commerce, Soft...|[https://www.vayu...|
|       Titan Capital|\n              V...|[https://www.link...|\n            Tit...|[Software, Intern...|[https://www.flic...|
|      Blume Ventures|\n              V...|[http://www.linke...|\n            Bac...|[Software, Inform...|[https://wiom.in ...|
|          Kunal Shah|\n              A...|[https://www.link...|\n            Kun...|[Financial Servic..

In [5]:
# Adding "id" column to dataframe
temp_df = df.rdd.zipWithIndex().toDF()
df = temp_df.select(col("_2").alias("id"), col("_1.*"))
df.show(n=10)

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|                name|                meta|               links|                desc|               focus|portfolio_highlights|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|Inflection Point ...|\n              V...|[https://www.link...|\n            Inf...|[Artificial Intel...|[https://kisah.in...|
|  1|        LVX Ventures|\n              V...|[http://www.linke...|\n            Con...|[E-Commerce, Soft...|[https://www.vayu...|
|  2|       Titan Capital|\n              V...|[https://www.link...|\n            Tit...|[Software, Intern...|[https://www.flic...|
|  3|      Blume Ventures|\n              V...|[http://www.linke...|\n            Bac...|[Software, Inform...|[https://wiom.in ...|
|  4|          Kunal Shah|\n              A...|[https://www.link...|\n      

In [6]:
# Removing spaces and "\n" from meta column
df = df.withColumn("meta", regexp_replace(df.meta, r"(\s{2,})|\\n", ""))
df.select("meta").show(truncate=False)

# Splitting meta column into sub-categories
df = df.withColumns({
    "type": trim(split(df.meta, "·").getItem(0)),
    "location": trim(split(df.meta, "·").getItem(1)),
    "investments_last_year": when(size(split(df.meta, "·"))>=3, trim(split(df.meta, "·").getItem(2))).otherwise(None),
})

# Extracting only number of investments in investments_last year
df = df.withColumn("investments_last_year", regexp_extract(df.investments_last_year, r"(.*(?=(\sinvestments)))", 1))

df.select("type", "location", "investments_last_year").show(truncate=False)

+------------------------------------------------------------------------------------+
|meta                                                                                |
+------------------------------------------------------------------------------------+
|VC Fund ·Gurgaon, Haryana, India ·39 investments in the past 12 months              |
|VC Fund ·Singapore, Central Region, Singapore ·16 investments in the past 12 months |
|VC Fund ·Gurgaon, Haryana, India ·17 investments in the past 12 months              |
|VC Fund ·Mumbai, Maharashtra, India ·36 investments in the past 12 months           |
|Angel ·Bengaluru, Karnataka, India ·11 investments in the past 12 months            |
|VC Fund ·New Delhi, Delhi, India ·15 investments in the past 12 months              |
|VC Fund ·Mumbai, Maharashtra, India ·7 investments in the past 12 months            |
|VC Fund ·Mumbai, Maharashtra, India                                                 |
|VC Fund ·Bengaluru, Karnataka, India ·52 i

In [7]:
# Splitting links column into sub-categories
df.select("links").show(truncate=135)
@udf
def extract_facebook_link(links):
    for link in links:
        if "facebook" in link:
            return link
@udf
def extract_twitter_link(links):
    for link in links:
        if "twitter" in link:
            return link
@udf
def extract_linkedin_link(links):
    for link in links:
        if "linkedin" in link:
            return link
@udf
def extract_website_link(links):
    for link in links:
        if "facebook" not in link and "twitter" not in link and "linkedin" not in link :
            return link
    
df = df.withColumns({
    "linkedin": extract_linkedin_link(df.links),
    "twitter": extract_twitter_link(df.links),
    "facebook": extract_facebook_link(df.links),
    "website": extract_website_link(df.links)
})
df.select("linkedin", "facebook", "twitter", "website").show(truncate=33)

+---------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                  links|
+---------------------------------------------------------------------------------------------------------------------------------------+
|[https://www.linkedin.com/company/ipventures/, https://twitter.com/ipventures_in, https://ipventures.in/, https://www.facebook.com/i...|
|[http://www.linkedin.com/company/letsventure, http://twitter.com/letsventurein, http://letsventure.com, http://www.facebook.com/Lets...|
|                      [https://www.linkedin.com/company/titan-capital-vc/, https://twitter.com/TitanCapitalVC, https://titancapital.vc]|
|[http://www.linkedin.com/company/blume-venture-advisors, http://twitter.com/BlumeVentures, http://blume.vc/, http://www.facebook.com...|
|                             [htt

+---------------------------------+---------------------------------+---------------------------------+---------------------------------+
|                         linkedin|                         facebook|                          twitter|                          website|
+---------------------------------+---------------------------------+---------------------------------+---------------------------------+
|https://www.linkedin.com/compa...|https://www.facebook.com/ipven...|https://twitter.com/ipventures_in|           https://ipventures.in/|
|http://www.linkedin.com/compan...|http://www.facebook.com/Letsve...| http://twitter.com/letsventurein|           http://letsventure.com|
|https://www.linkedin.com/compa...|                             NULL|https://twitter.com/TitanCapit...|          https://titancapital.vc|
|http://www.linkedin.com/compan...|http://www.facebook.com/Blumev...| http://twitter.com/BlumeVentures|                 http://blume.vc/|
|https://www.linkedin.com/in/ku...

In [8]:
# Removing spaces and "\n" from desc column
df.select("desc").show(truncate=False, n=1)
df = df.withColumn("desc", regexp_replace(df.desc, r"(\s{2,})|\\n", ""))
df.select("desc").show(truncate=False, n=1)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|desc                                                                                                                                                                                                                                                                                                                               

In [9]:
# Splitting investment_focus into sub-categories
df.select("focus").show(truncate=False, n=10)
df = df.withColumns({
    "sector": df.focus.getItem(0),
    "stage": df.focus.getItem(1),
    "region": df.focus.getItem(2),
})
df.select("sector", "stage", "region").show(truncate=False, n=10)

+-------------------------------------------------------------------------------------------------------------------+
|focus                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------+
|[Artificial Intelligence, E-Commerce, Health Care, Seed, Pre-Seed, Series A, India, United States, Singapore]      |
|[E-Commerce, Software, Internet, Seed, Series A, Angel, India, United States, Italy]                               |
|[Software, Internet, Information Technology, Seed, Pre-Seed, Series A, India, United States, Singapore]            |
|[Software, Information Technology, Internet, Seed, Series A, Series B, India, United States, Singapore]            |
|[Financial Services, FinTech, Internet, Seed, Pre-Seed, Series A, India, United States, Singapore]                 |
|[Software, Health Care, SaaS, Seed, Series A, Angel, In

In [10]:
# Flattening each value of sector, stage, region into multiple rows
print("Count of each 'name' before flattening: ")
df.groupBy("name").count().show()

df = df.withColumn("sector", split(df.sector, ","))
df = df.withColumn("sector", explode(df.sector))

df = df.withColumn("stage", split(df.stage, ","))
df = df.withColumn("stage", explode(df.stage))

df = df.withColumn("region", split(df.region, ","))
df = df.withColumn("region", explode(df.region))

print("Count of each 'name' after flattening ~27(3*3*3): ") #Because each sector, stage, region has around 3 values
df.groupBy("name").count().show()

df.select("name", "sector", "stage", "region").show()

Count of each 'name' before flattening: 


+--------------------+-----+
|                name|count|
+--------------------+-----+
|Inflection Point ...|    1|
|           IAN Group|    1|
|      Blume Ventures|    1|
|        LVX Ventures|    1|
|       Titan Capital|    1|
|          Kunal Shah|    1|
|       Mumbai Angels|    1|
|      India Quotient|    1|
|             100X.VC|    1|
|               Accel|    1|
|    Peak XV Partners|    1|
|      Better Capital|    1|
|        ah! Ventures|    1|
|       3one4 Capital|    1|
|        First Cheque|    1|
|     Kalaari Capital|    1|
|            Omnivore|    1|
|                 Z47|    1|
|Nexus Venture Par...|    1|
|             BEENEXT|    1|
+--------------------+-----+
only showing top 20 rows
Count of each 'name' after flattening ~27(3*3*3): 
+--------------------+-----+
|                name|count|
+--------------------+-----+
|Inflection Point ...|   27|
|           IAN Group|   27|
|      Blume Ventures|   27|
|        LVX Ventures|   27|
|       Titan Capital|   

In [11]:
# Flatten each value of portfolio_highlights into multiple rows
print("Count of each 'name' before flattening: ")
df.groupBy("name").count().show()

df = df.withColumn("portfolio_highlights", explode(df.portfolio_highlights))

print("Count of each 'name' after flattening ~81(27*3): ") #Because each portfolio_highlights has around 3 values
df.groupBy("name").count().show()

Count of each 'name' before flattening: 
+--------------------+-----+
|                name|count|
+--------------------+-----+
|Inflection Point ...|   27|
|           IAN Group|   27|
|      Blume Ventures|   27|
|        LVX Ventures|   27|
|       Titan Capital|   27|
|          Kunal Shah|   27|
|       Mumbai Angels|   27|
|      India Quotient|   27|
|             100X.VC|   18|
|               Accel|   27|
|    Peak XV Partners|   27|
|      Better Capital|   27|
|        ah! Ventures|   27|
|       3one4 Capital|   27|
|        First Cheque|   18|
|     Kalaari Capital|   27|
|            Omnivore|   18|
|                 Z47|   27|
|Nexus Venture Par...|   27|
|             BEENEXT|   27|
+--------------------+-----+
only showing top 20 rows
Count of each 'name' after flattening ~81(27*3): 
+--------------------+-----+
|                name|count|
+--------------------+-----+
|Inflection Point ...|   81|
|           IAN Group|   81|
|      Blume Ventures|   81|
|        LVX V

In [12]:
# Splitting portfolio_highlights into sub-categories
df.select("portfolio_highlights").show(truncate=135)
df = df.withColumns({
    "portfolio_name": btrim(split(df.portfolio_highlights, "—").getItem(1), trim=lit(" \n")),
    "portfolio_link": trim(split(df.portfolio_highlights, "—").getItem(0)),
    "portfolio_desc": btrim(split(df.portfolio_highlights, "—").getItem(2), trim=lit(" \n"))
})
df.select("name", "portfolio_name", "portfolio_link", "portfolio_desc").show(truncate=65)

+---------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   portfolio_highlights|
+---------------------------------------------------------------------------------------------------------------------------------------+
|https://kisah.in/ — \nKisah — Elevate your style with Kisah, the leading men's ethnic wear brand blending Gen Z flair with timeless ...|
|https://www.iombio.com — \nIom — Iom Bioworks is an Indo Japanese microbiome-based life sciences company. Iom uses DNA sequencing, b...|
|https://cleanfanatics.com — \nClean Fanatics — Clean Fanatics offers expert house cleaning, painting, pest control, and civil servic...|
|https://kisah.in/ — \nKisah — Elevate your style with Kisah, the leading men's ethnic wear brand blending Gen Z flair with timeless ...|
|https://www.iombio.com — \nIom — 

In [13]:
# Offloading desc & portfolio_desc columns into another table as they have long texts and are redundant
df_desc = df.select(col("id").alias("investor_id"), "desc").distinct().sort("investor_id")
df_desc.show(truncate=120)
df_portfolio_desc = df.select(col("id").alias("investor_id"), "portfolio_desc").distinct().sort("investor_id")
df_portfolio_desc.show(truncate=120)

+-----------+------------------------------------------------------------------------------------------------------------------------+
|investor_id|                                                                                                                    desc|
+-----------+------------------------------------------------------------------------------------------------------------------------+
|          0|Inflection Point Ventures, with over 3300 CXOs and professionals, HNIs and Family Offices, is India’s leading angel n...|
|          1|Connect with 10,000 plus angel investors. Raise funding seamlessly. Find startups to invest in. Easy to use & seamles...|
|          2|Titan Capital invests behind world class entrepreneurs looking to create wide-scale positive impact on India and the ...|
|          3|                                             Backing the next wave of revolutionary founders, from India, for the world.|
|          4|                                          

In [14]:
# Final investors table:
df = df.select(
    "id",
    "name",
    "type",
    "location",
    "investments_last_year",
    "sector",
    "stage",
    "region",
    "portfolio_name",
    "portfolio_link",
    "linkedin",
    "facebook",
    "twitter",
    "website",
)
df.show(truncate=6)

+---+------+------+--------+---------------------+------+------+------+--------------+--------------+--------+--------+-------+-------+
| id|  name|  type|location|investments_last_year|sector| stage|region|portfolio_name|portfolio_link|linkedin|facebook|twitter|website|
+---+------+------+--------+---------------------+------+------+------+--------------+--------------+--------+--------+-------+-------+
|  0|Inf...|VC ...|  Gur...|                   39|Art...|  Seed| India|         Kisah|        htt...|  htt...|  htt...| htt...| htt...|
|  0|Inf...|VC ...|  Gur...|                   39|Art...|  Seed| India|           Iom|        htt...|  htt...|  htt...| htt...| htt...|
|  0|Inf...|VC ...|  Gur...|                   39|Art...|  Seed| India|        Cle...|        htt...|  htt...|  htt...| htt...| htt...|
|  0|Inf...|VC ...|  Gur...|                   39|Art...|  Seed| Un...|         Kisah|        htt...|  htt...|  htt...| htt...| htt...|
|  0|Inf...|VC ...|  Gur...|                   3

In [15]:
# Load data

df.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("s3a://elasticbeanstalk-ap-south-1-576190372469/myc/silver/shizune_investors")

df_desc.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("s3a://elasticbeanstalk-ap-south-1-576190372469/myc/silver/shizune_investors_desc")

df_portfolio_desc.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("s3a://elasticbeanstalk-ap-south-1-576190372469/myc/silver/shizune_investors_portfolio_desc")


25/09/06 19:26:47 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
                                                                                

In [16]:
# Read & display data to test:
spark.read.parquet("s3a://elasticbeanstalk-ap-south-1-576190372469/myc/silver/shizune_investors").show()
spark.read.parquet("s3a://elasticbeanstalk-ap-south-1-576190372469/myc/silver/shizune_investors_desc").show()
spark.read.parquet("s3a://elasticbeanstalk-ap-south-1-576190372469/myc/silver/shizune_investors_portfolio_desc").show()

+---+--------------------+-------+--------------------+---------------------+--------------------+--------------+--------------+----------------+--------------------+--------------------+--------------------+--------------------+------------------+
| id|                name|   type|            location|investments_last_year|              sector|         stage|        region|  portfolio_name|      portfolio_link|            linkedin|            facebook|             twitter|           website|
+---+--------------------+-------+--------------------+---------------------+--------------------+--------------+--------------+----------------+--------------------+--------------------+--------------------+--------------------+------------------+
| 42|YourNest Venture ...|VC Fund|Gurgaon, Haryana,...|                   11|Artificial Intell...|          Seed|         India|     Map My Crop|https://www.mapmy...|https://www.linke...|https://www.faceb...|https://twitter.c...|http://yournest.in|
| 42